In [2]:
import pandas as pd
import requests
from io import BytesIO
import gzip

In [13]:
title_basics = pd.read_csv("C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/title.basics.tsv.gz", sep='\t', low_memory=False)

In [6]:
title_ratings = pd.read_csv("C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/title.ratings.tsv.gz", sep='\t', low_memory=False)

In [8]:
title_akas = pd.read_csv("C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/title.akas.tsv.gz", sep='\t', low_memory=False)

In [14]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [15]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10248781 entries, 0 to 10248780
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 703.7+ MB


In [7]:
title_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2002
1,tt0000002,5.8,269
2,tt0000003,6.5,1893
3,tt0000004,5.5,178
4,tt0000005,6.2,2678


In [12]:
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360168 entries, 0 to 1360167
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1360168 non-null  object 
 1   averageRating  1360168 non-null  float64
 2   numVotes       1360168 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.1+ MB


In [9]:
title_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [11]:
title_akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37517220 entries, 0 to 37517219
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [21]:
# Filter movies based on the specified requirements
# 1. Exclude any movie with missing values for genre or runtime
title_basics = title_basics.dropna(subset=['genres', 'runtimeMinutes'])

In [22]:
# 2. Include only full-length movies (titleType = "movie")
title_basics = title_basics[title_basics['titleType'] == 'movie']

In [23]:
# 3. Include only fictional movies (not from documentary genre)
title_basics = title_basics[~title_basics['genres'].str.contains('Documentary', case=False, na=False)]

In [24]:
# 4. Include only movies that were released 2000 - 2021 (include 2000 and 2021)
title_basics['startYear'] = pd.to_numeric(title_basics['startYear'], errors='coerce')
title_basics = title_basics[(title_basics['startYear'] >= 2000) & (title_basics['startYear'] <= 2021)]
# 5. Include only movies that were released in the United States
title_akas = title_akas[title_akas['region'] == 'US']
us_movie_ids = title_akas['titleId'].unique()
title_basics = title_basics[title_basics['tconst'].isin(us_movie_ids)]

In [25]:
# Display information about the remaining movies
print("Number of movies after filtering:", len(title_basics))
print("Title Basics Info:")
print(title_basics.info())

Number of movies after filtering: 97815
Title Basics Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 97815 entries, 34800 to 10248579
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          97815 non-null  object 
 1   titleType       97815 non-null  object 
 2   primaryTitle    97815 non-null  object 
 3   originalTitle   97815 non-null  object 
 4   isAdult         97815 non-null  object 
 5   startYear       97815 non-null  float64
 6   endYear         97815 non-null  object 
 7   runtimeMinutes  97815 non-null  object 
 8   genres          97815 non-null  object 
dtypes: float64(1), object(8)
memory usage: 7.5+ MB
None


In [26]:
# Save the filtered dataframes to compressed CSV files in the "Data/" folder
data_folder = "C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/"
title_basics.to_csv(f"{data_folder}filtered_title_basics.csv.gz", sep='\t', compression='gzip', index=False)

In [28]:
# Display information about the Title Akas
print("Number of movies after filtering:", len(title_akas))
print("Title Akas Info:")
print(title_akas.info())

Number of movies after filtering: 1472163
Title Akas Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472163 entries, 5 to 37516964
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1472163 non-null  object
 1   ordering         1472163 non-null  int64 
 2   title            1472163 non-null  object
 3   region           1472163 non-null  object
 4   language         1472163 non-null  object
 5   types            1472163 non-null  object
 6   attributes       1472163 non-null  object
 7   isOriginalTitle  1472163 non-null  object
dtypes: int64(1), object(7)
memory usage: 101.1+ MB
None


In [29]:
# Display information about the Title Ratings
print("Number of movies after filtering:", len(title_ratings))
print("Title Ratings Info:")
print(title_ratings.info())

Number of movies after filtering: 1360168
Title Ratings Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1360168 entries, 0 to 1360167
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1360168 non-null  object 
 1   averageRating  1360168 non-null  float64
 2   numVotes       1360168 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 31.1+ MB
None


In [30]:
# Save the filtered dataframes to compressed CSV files in the "Data/" folder
data_folder = "C:/Users/MiPillay1/OneDrive - MMI HOLDINGS LTD/Documents/GitHub/Project_3-Data_Enrichment/Data/"
title_akas.to_csv(f"{data_folder}filtered_title_akas.csv.gz", sep='\t', compression='gzip', index=False)